# 1. Import the necessary libraries for you to be able to query 4square API

In [6]:
import requests
import json
import pandas as pd
from getpass import getpass
from pymongo import MongoClient
import time

In [7]:
import os
from dotenv import load_dotenv
load_dotenv() # load_env

True

In [8]:
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [161]:
where = "Calle de Pamplona, 96, 08018 Barcelona, Spain"

In [162]:
def gets_coordinates_from_string (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    
    try:
        return res.json()
    
    except:
        time.sleep(10)
        gets_coordinates_from_string (where)

In [163]:
res = gets_coordinates_from_string("Calle de Pamplona, 96, 08018 Barcelona, Spain")
res

{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.8'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [164]:
lat, lon = res["latt"], res["longt"]

In [165]:
lat

'41.39780'

In [166]:
lon

'2.19035'

In [167]:
def requests_for_foursquare (query, lat, lon, radius=500, limit=10):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [168]:
results = requests_for_foursquare ("coffee", lat, lon, radius=600, limit=10)
results

{'results': [{'fsq_id': '530c5e6911d2be0b8d22e27a',
   'categories': [{'id': 13034,
     'name': 'Café',
     'short_name': 'Café',
     'plural_name': 'Cafés',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 135,
   'geocodes': {'main': {'latitude': 41.396937, 'longitude': 2.191519},
    'roof': {'latitude': 41.396937, 'longitude': 2.191519}},
   'link': '/v3/places/530c5e6911d2be0b8d22e27a',
   'location': {'address': 'Calle

In [169]:
#This function is for testing
def extracting_from_one_element (one):

    name = one["name"]
    address = one["location"]["address"]
    distance = one["distance"]

    
    small_dict = {
    "name": name,
    "address": address,
    "distance": distance,
}
    return small_dict

In [170]:
extracting_from_one_element (response.json()["results"][4])

{'name': 'Eva-2', 'address': 'Bolivia, 21', 'distance': 445}

In [177]:
def transform_request_into_df (results):
    list_of_dictionaries = [extracting_from_one_element (element) for element in results["results"]]
    df = pd.DataFrame(list_of_dictionaries)
    
    df.sort_values(by='distance', inplace = True)
    return df

In [178]:
transform_request_into_df(results)

,name,address,distance
8,Espacio 88,"Carrer Pamplona, 88",124
0,SKYE Coffee Co,"Calle Pamplona, 88",135
1,Syra Coffee Poblenou,"Calle de Pujades, 100",342
2,Espai Joliu,"Carrer Badajoz, 95",399
9,Granja Mabel,"Marina, 114-116",418
4,Eva-2,"Bolivia, 21",439
3,Nømad Roaster's Home,"Calle de Pujades, 95",450
5,Mare Meva,"C. de Bolívia, 31",487
6,Cafeteria Bar BAU,"Pujades, 118",493
7,El Bar,"Ribes, 46",572


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [183]:

url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=41.3977461%2C2.187669&radius=500&limit=5"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

results =response.json()
results

{'results': [{'fsq_id': '530c5e6911d2be0b8d22e27a',
   'categories': [{'id': 13034,
     'name': 'Café',
     'short_name': 'Café',
     'plural_name': 'Cafés',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 320,
   'geocodes': {'main': {'latitude': 41.396937, 'longitude': 2.191519},
    'roof': {'latitude': 41.396937, 'longitude': 2.191519}},
   'link': '/v3/places/530c5e6911d2be0b8d22e27a',
   'location': {'address': 'Calle

In [184]:
five_coffee_shops= transform_request_into_df(results)
five_coffee_shops

,name,address,distance
0,SKYE Coffee Co,"Calle Pamplona, 88",320
1,Three Marks Coffee,"Ausiàs Marc, 151",354
3,El Bar,"Ribes, 46",364
2,El Cafè Blau,"Casp, 192",430
4,Eva-2,"Bolivia, 21",445


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [1]:
def foursquare_places(venue, coordinates):
    lat, lon = str(coordinates).split(',')
    res = requests_for_foursquare (venue, lat, lon, radius = 500, sort = "DISTANCE", limit = 10)
    
    try:
        data = res['results']
        results = []
        
        for i in data:
            name = i['name']
            location = i['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [ ]:
# your code here
pd.DataFrame(res).to_json('data/coffe_shops.json')

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here

client = MongoClient("localhost:27017")
db = client["ironhack"]
collection = db['ironcoffee']
for coffee in five_coffee_shops:
    collection.insert_one(coffee)